In [0]:
from pyspark.sql import functions as f

In [0]:
%run ./../../ingestion/structs/nb_schemas

In [0]:
# Parâmetros de entrada para a tabela Silver
src_catalog_name    = 'silver'
src_schema_name     = 'refined'
src_table_name      = 'global_superstore'

# Parâmetros da camada Gold
gold_catalog_name   = 'gold'
gold_schema_name    = 'analytics'
gold_table_prefix   = 'global_superstore'

gold_parquet_tables = '/Volumes/gold/analytics/parquets_files'

In [0]:

table_path = f"{src_catalog_name}.{src_schema_name}.{src_table_name}"
spark.sql(f"CREATE CATALOG IF NOT EXISTS {gold_catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_catalog_name}.{gold_schema_name}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {gold_catalog_name}.{gold_schema_name}.parquets_files")
df = spark.read.table(table_path)

In [0]:
df = spark.read.table(table_path)

#### Consulta: Métricas Anuais por produto

In [0]:
# Agregue os dados para obter estatísticas de vendas, por exemplo, o total de vendas por
# produto ou por categoria

df_category = (
    df
    .groupBy('category', f.substring(f.col('order_date'), 1, 7).alias('month'))
    .agg(
        f.sum("sales").alias("total_sales")
    )
)

(
    df_category
    .write
    .format("delta")
    .mode("overwrite")
    .option('overwriteSchema', True)
    .saveAsTable(f"{gold_catalog_name}.{gold_schema_name}.{gold_table_prefix}_category")
)